# Action Items:
### 1) Incorportate flash attn
### 2) Diferent experiment for quantization and LORA
### 3) An Eval script to eval on [Benchmark](https://github.com/meta-llama/PurpleLlama/tree/main/CybersecurityBenchmarks)

In [ ]:
!pip install datasets -Uqq
!pip install trl -Uqq

In [ ]:
from datasets import load_dataset
from trl import DPOTrainer, DPOConfig
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)

In [ ]:
dataset = load_dataset("CyberNative/Code_Vulnerability_Security_DPO")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
dataset['train']['question'][0]

"Write a c++ code that defines a function named 'copyString' which takes two character pointers as arguments. The first pointer points to the source string and the second one points to the destination string. This function should copy the content of the source string into the destination string. In the main function, declare two character arrays, one as the source string and another as the destination string. Call the 'copyString' function with these two arrays as arguments."

In [ ]:
model_name = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'

In [ ]:
# !pip install flash-attn --no-build-isolation

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
dpo_config = DPOConfig(
    beta=0.1,
    learning_rate=5e-5,
    per_device_train_batch_size=64,
    max_length=1024,
    max_prompt_length=512,
    gradient_accumulation_steps=2,
    logging_steps=10,
    output_dir="dpo-trained-model",
    optim="paged_adamw_32bit"
)


In [ ]:
dpo_trainer = DPOTrainer(
    model=model,
    ref_model=None,
    args=dpo_config,
    train_dataset=dataset["train"],
    tokenizer=tokenizer
)

<ipython-input-10-4e972e4f8fdc>:1: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `DPOTrainer.__init__`. Use `processing_class` instead.
  dpo_trainer = DPOTrainer(


In [ ]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.0 MB/s eta 0:00:00


In [ ]:
# Not enough RAM
dpo_trainer.train()

ERROR:bitsandbytes.cextension:Could not load bitsandbytes native library: /lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.32' not found (required by /usr/local/lib/python3.11/dist-packages/bitsandbytes/libbitsandbytes_cpu.so)
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/bitsandbytes/cextension.py", line 85, in <module>
    lib = get_native_library()
          ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/bitsandbytes/cextension.py", line 72, in get_native_library
    dll = ct.cdll.LoadLibrary(str(binary_path))
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 454, in LoadLibrary
    return self._dlltype(name)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 376, in __init__
    self._handle = _dlopen(self._name, mode)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: /lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: itsanirudhjoshi (itsanirudhjoshi-carnegie-mellon-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
